<a href="https://colab.research.google.com/github/JoanneBurke/Data-Analysis/blob/master/Attempting_TITLE_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attemtpting text clustering using doc2vec

The idea behind this is to potentially find the 'odd texts' out. Hopefully once the elements have been clustered, a potential thing that could be done is topic modelling on each cluster and see what that gives - maybe by insepction this may give show the documents which do not tackle land conflcits

In [0]:
!pip install news-please

In [0]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/Omdena/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Omdena


In [0]:
!tar -xvzf environmental-conflict-tracker/enwiki_dbow.tgz

enwiki_dbow/
enwiki_dbow/doc2vec.bin.syn1neg.npy
enwiki_dbow/doc2vec.bin.syn0.npy
enwiki_dbow/doc2vec.bin


In [0]:
import pickle
import pandas as pd
from newsplease import NewsPlease
import gensim

In [0]:
!ls environmental-conflict-tracker/data/

gold-standard  metadata  policy  reference  texts  urls


In [0]:
import gensim.models

model = 'enwiki_dbow/doc2vec.bin'
output_file = 'environmental-conflict-tracker/data/test_vectors.txt'

Now I need to extract the text documents

In [0]:
import pickle
import pandas as pd
from newsplease import NewsPlease
import numpy as np
import os

def load_obj(month):
    month = str(month).zfill(2)
    month_texts = []
    path = 'environmental-conflict-tracker/data/texts/{}/'.format(month)
    # print('bob')
    for dirpath, dirnames, filenames in os.walk(path):
      # print(filenames)
      for file_ in filenames:
        # print(file_)
        with open(path + file_, "rb") as f:
          # print(pickle.load(f).text)
          try:
            title_ = pickle.load(f).title
            # print(title_)
            if (title_ == 'Page Not Found'):
              continue
            month_texts.append(title_.split())
            # print(month_texts)
          except:
            # print('bad')
            continue
    with open(month + "_title.txt", "wb") as fb:
      pickle.dump(month_texts, fb)
      print('saved ' + month)
    return month_texts

In [0]:
!ls

 01_title.txt   06_title.txt   all_texts.txt
 01.txt         06.txt	       cluster_topics.pkl
 02_title.txt   07_title.txt   elbow_5.png
 02.txt         07.txt	       elbow.png
 03_title.txt   08_title.txt   environmental-conflict-tracker
 03.txt         08.txt	       enwiki_dbow
 04_title.txt   09.txt	      'Getting Started.ipynb'
 04.txt         10.txt	       stanford-corenlp-full-2018-10-05
 05_title.txt   11.txt	       vectors.pkl
 05.txt         12.txt


In [0]:
import numpy as np
months = [str(el).zfill(2) for el in np.arange(13)[1:]]

In [0]:
for el in months:
  load_obj(el)

saved 01
saved 02
saved 03
saved 04
saved 05
saved 06
saved 07
saved 08
saved 09
saved 10
saved 11
saved 12


In [0]:
with open("01_title.txt", "rb") as f:
  month_1 = pickle.load(f)

In [0]:
month_1

In [0]:
path = 'environmental-conflict-tracker/data/texts/01/'
for dirpath, dirnames, filenames in os.walk(path):
  print(len(filenames))

6627


What I had noticed previously was that for some reason some had text set to None. This should also ideally be flagged and removed. I find it weird

In [0]:
!pwd

/content


In [0]:
all_docs = None
for m in months:
  with open("{}_title.txt".format(m), "rb") as f:
    month_texts = pickle.load(f)
    if all_docs is None:
      all_docs = month_texts
    else:
      all_docs += month_texts

In [0]:
len(all_docs)

64877

In [0]:
month_1

In [0]:
with open("all_titles.txt", "wb") as fb:
      pickle.dump(all_docs, fb)

# Attempt LDA on all the titles

In [0]:
all_docs = [' '.join(x) for x in all_docs]

In [0]:
!pip install preprocessing


In [0]:
'a b c'.split()

['a', 'b', 'c']

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#remove stopwords, punctuation etc
import string
from nltk.corpus import stopwords


stop_words = set(stopwords.words('english'))

for i,title in enumerate(all_docs):
  title = title.lower()
  title = title.translate(str.maketrans('', '', string.punctuation))
  title = ' '.join([w for w in title.split() if w not in stop_words])
  all_docs[i] = title



In [0]:
all_docs[i]

'rains fail chennai city prepared face another waterscarce summer'

In [0]:
with open("all_titles_preprocessed.txt", "wb") as fb:
      pickle.dump(all_docs, fb)

In [0]:
tf_vectorizer = LemmaCountVectorizer(max_df=0.95,
                                   min_df=2,
                                   stop_words='english',
                                   decode_error='ignore')
tf = tf_vectorizer.fit_transform(all_docs)
lda = LatentDirichletAllocation(n_components=11, max_iter=5,
                              learning_method = 'online',
                              learning_offset = 50.,
                              random_state = 0)
lda.fit(tf)
n_top_words = 5
print("\nTopics in LDA model: ")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model: 

Topic #0:protest violence kerala demand hold

Topic #1:india new development caste news

Topic #2:court case killed village bulandshahr

Topic #3:farmer killing modi govt pm

Topic #4:road r bjp minister project

Topic #5:people pakistan mp issue agustawestland

Topic #6:say man kill land dead

Topic #7:delhi farm loan year day

Topic #8:airport dead killed arrested held

Topic #9:protest chidambaram water cow chief

Topic #10:killed water indian state attack


#Attempting to vectorize them with USE (UNIVERSAL SENTENCE ENCODER)

In [0]:
import tensorflow as tf
import tensorflow_hub as tf_hub

In [0]:
model_url = 'https://tfhub.dev/google/universal-sentence-encoder/2'
embed = tf_hub.Module(model_url)

In [0]:
with open('all_titles.txt', 'rb') as f:
  all_docs = pickle.load(f)

In [0]:
all_docs = [' '.join(t) for t in all_docs]

In [0]:
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(all_docs))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [0]:
 df = pd.DataFrame({'titles': all_docs, 'embed': message_embeddings.tolist()})

In [0]:
df.head(3)

,titles,embed
0,Lawyer killed as truck hits car,"[-0.020477019250392914, -0.06466207653284073, ..."
1,Page not Found,"[0.008759897202253342, -0.016028402373194695, ..."
2,Telangana government makes harsh language a co...,"[0.014865786768496037, -0.06467095762491226, -..."


In [0]:
df.to_pickle('title_vectors.pkl')

In [0]:
import numpy as np
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20,init='k-means++',max_iter=300,n_init=10,random_state=0)
kmeans.fit(df['embed'].values.tolist())

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=20, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [0]:
labels = kmeans.labels_

In [0]:
df['labels'] = labels


In [0]:
df[df['labels'] == 3]

In [0]:
df.to_pickle('title_vectors.pkl')

In [0]:
df.head()

,titles,embed,labels
0,Lawyer killed as truck hits car,"[-0.020477019250392914, -0.06466207653284073, ...",11
1,Page not Found,"[0.008759897202253342, -0.016028402373194695, ...",19
2,Telangana government makes harsh language a co...,"[0.014865786768496037, -0.06467095762491226, -...",7
3,Nature provides artistic inspiration for retir...,"[0.0431208610534668, -0.04448771849274635, -0....",6
4,18 arrested over Gurugram school bus attack du...,"[-0.044236116111278534, -0.08111345022916794, ...",0


In [0]:
df.groupby('labels')['titles'].count()

labels
0     1358
1     5475
2     3335
3     2638
4     2760
5     2907
6     2983
7     5255
8      485
9     2890
10     539
11    4004
12    5039
13    6284
14    3659
15    3454
16    3694
17    4028
18    3573
19     517
Name: titles, dtype: int64

# Topics of Titles

In [0]:
df[df['labels'] == 8]

,titles,embed,labels
36,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
39,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
81,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
91,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
93,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
127,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
128,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
243,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
436,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8
718,Welcome To IANS Live,"[-0.0005704158684238791, -0.043604832142591476...",8


In [0]:
from collections import defaultdict

tf_vectorizer = LemmaCountVectorizer(max_df=0.95,
                                   min_df=2,
                                   stop_words='english',
                                   decode_error='ignore')

def get_top_words(model, feature_names, n_top_words):

  d = defaultdict(list)

  for index, topic in enumerate(model.components_):
      message = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1 :-1]])
      d[index].append(message)
  return d

all_topics = defaultdict(list)

for i in range(20):
  if (i==8):
    continue
  tf_ = tf_vectorizer.fit_transform(df[df['labels'] == i]['titles'].values)
  lda = LatentDirichletAllocation(n_components=7, max_iter=5,
                              learning_method = 'online',
                              learning_offset = 50.,
                              random_state = 0)
  lda.fit(tf_)
  n_top_words = 10
  print("\nTopics in LDA model: ")
  tf_feature_names = tf_vectorizer.get_feature_names()
  d = get_top_words(lda, tf_feature_names, n_top_words)

  for k,v in d.items():
    all_topics[k].append(v)

df_topic = pd.DataFrame({})

for k, v in all_topics.items():
  df_topic['Topic ' + str(k)] = v


Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 

Topics in LDA model: 


In [0]:
df_topic.iloc[8].values

array([list(['farmer farm loan waiver demand new ban hit congress r']),
       list(['cow cattle land village help green farming held business industry']),
       list(['farmer land crop govt farming india protest organic end support']),
       list(['farmer price income march day milk villager 10 want change']),
       list(['forest farmer mining agriculture plan govt minister say illegal coal']),
       list(['farmer government indian conservation development rally wildlife dairy sugar pesticide']),
       list(['agriculture farm water export sector need ministry policy stop president'])],
      dtype=object)

In [0]:
df_topic.to_pickle('title_topic.pkl')

In [0]:
!ls

 01_title.txt   07.txt			      all_titles.txt
 01.txt         08_title.txt		      cluster_topics.pkl
 02_title.txt   08.txt			      elbow_5.png
 02.txt         09_title.txt		      elbow.png
 03_title.txt   09.txt			      environmental-conflict-tracker
 03.txt         10_title.txt		      enwiki_dbow
 04_title.txt   10.txt			     'Getting Started.ipynb'
 04.txt         11_title.txt		      stanford-corenlp-full-2018-10-05
 05_title.txt   11.txt			      title_topic.pkl
 05.txt         12_title.txt		      title_vectors.pkl
 06_title.txt   12.txt			      vectors.pkl
 06.txt         all_texts.txt
 07_title.txt   all_titles_preprocessed.txt


In [0]:
import pandas as pd
vec = pd.read_pickle('title_vectors.pkl')
topic = pd.read_pickle('title_topic.pkl')

In [0]:
topic.head()

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6
0,[nun bishop jalandhar link terror mulakkal pri...,[student university protest teacher delhi kill...,[student protest school delhi parent jnu class...,[senior nsui mu reinstatement aiims free fake ...,[student protest college caste school govt adm...,[school shut remain ghaziabad protest bandh fa...,[protest school village institute ministry emi...
1,[killed r violence bulandshahr say killing lak...,[killing man mumbai old delhi year held woman ...,[kill man killed dead wife woman india mob ind...,[girl cow case old killer india year rape vill...,[dead shot delhi police killed pradesh officer...,[family delhi kerala case dead member alleged ...,[maharashtra killing farmer gauri hc cbi benga...
2,[deadline accused jalandhar bulandshahr set ma...,[dead babu verify damage sinha judiciary convi...,[michel agustawestland christian middleman ind...,[land r case court settlement dispute year cro...,[cbi case probe court land say scam chief deal...,[court sc supreme judge examine case say verdi...,[chidambaram case court karti medium inx cbi e...
3,[road gadkari safety say expressway bengal nit...,[road bridge india rail longest mumbai constru...,[india launch new lakh thailand land rover tat...,[road delhi mumbai traffic airport development...,[r infrastructure cr road development water tr...,[road land project mumbai village train bullet...,[vehicle pollution electric ashok leyland fore...
4,[airport air india pilot nod force crash near ...,[airport flight gold mumbai landing delhi r ma...,[airport flight international kerala kannur op...,[airport passenger detained airline plane cisf...,[navy indian island rescue seek vizag meghalay...,[land air india airport aai launch order expan...,[airport aircraft runway hour mumbai shut high...


In [0]:
topic.iloc[15].values

array([list(['dead indian killed iraq shot 39 mosul body isi swaraj']),
       list(['kashmir indian killed army kill soldier militant civilian force troop']),
       list(['india kill rebel maoist near police killed anti 14 pakistan']),
       list(['attack killed terror year killing grenade injured amritsar suspected dead']),
       list(['pakistan killed india protest bsf killing jawan pak firing ceasefire']),
       list(['killed terrorist kashmir encounter jammu security clash force militant blast']),
       list(['killing kashmiri sikh condemns indian kashmir pakistan civilian assam massacre'])],
      dtype=object)